## Meta llama rational steps

In [5]:
from groq import Groq
from dotenv import dotenv_values

In [6]:
class meta_llama_rational_steps:

    def __init__(self, max_tokens=1200):
        CONFIG = dotenv_values("config/.env")

        self.client = Groq(api_key=CONFIG["GROQ_API_KEY"])
        self.model_name = CONFIG["MODEL_NAME"]
        self.max_tokens = max_tokens
    
    def question_steps_answer(self, question, system_prompt="Answer the following question"):
        response = self.client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": f"{system_prompt}"
                },
                {
                    "role": "user",
                    "content": f"{question}"
                }
            ],
            model=self.model_name,
            max_tokens=self.max_tokens
        )
        answer = response.choices[0].message.content
        all_steps = answer.split("\n\n")
        rational_steps = all_steps[:-1]
        final_answer = all_steps[-1]

        return rational_steps, final_answer

In [75]:
system_prompt = "Think step by step. Use a line break if and only if after finishing a step. The last line should be the final answer. For example: Do this \n Do that \n The answer is 3"
model_rational_steps = meta_llama_rational_steps()

In [ ]:
question="Find a word that relates the word 'novel' and the word 'hotel'"
rational_steps, final_answer = model_rational_steps.question_steps_answer(question=question, system_prompt=system_prompt)

## Steps classification

In [22]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from torch.nn.functional import cosine_similarity
import numpy as np
from scipy.stats import chi2

In [95]:
question_answer_train = pd.read_csv("data/question_answer_train.csv")
question_answer_test = pd.read_csv("data/question_answer_test.csv")

system_prompt = "Think step by step. Use a line break if and only if after finishing a step. The last line should be the final answer (be as concise as possible on the final answer!). For example: Do this \n Do that \n 4"

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model_embeddings = AutoModel.from_pretrained('distilbert-base-uncased')

In [48]:
question_answer_train

,problem,level,type,solution,answer
0,The United States Postal Service charges an ex...,Level 3,Prealgebra,We calculate the desired ratio for each envelo...,3
1,How many integers between 1000 and 2000 have a...,Level 4,Prealgebra,"A number with 15, 20 and 25 as factors must be...",3
2,"Given that $n$ is an integer and $0 < 4n <30$,...",Level 2,Prealgebra,"Dividing by $4$, we have $0<n<7\frac{1}{2}$. T...",28
3,How many integers between $100$ and $150$ have...,Level 4,Prealgebra,We will break up the problem into cases based ...,18
4,Regular pentagon $ABCDE$ and regular hexagon $...,Level 4,Prealgebra,We know that the sum of the degree measures of...,132
...,...,...,...,...,...
7493,Let $z$ be a complex number with $|z| = \sqrt{...,Level 5,Intermediate Algebra,"Let $z = x + yi,$ where $x$ and $y$ are real n...",4 \sqrt{2}
7494,Let $a$ and $b$ be real numbers. Find the max...,Level 4,Intermediate Algebra,"By the Cauchy-Schwarz Inequality,\n\[(a \cos \...",\sqrt{a^2 + b^2}
7495,"In the complex plane, $z,$ $z^2,$ $z^3$ form, ...",Level 5,Intermediate Algebra,"First, consider the case where $z$ is between ...","\frac{5}{8}, 2, 10"
7496,There is a smallest positive real number $a$ s...,Level 5,Intermediate Algebra,"Let $r,$ $s,$ $t$ be the real roots, so\n\[r^3...",9


In [43]:
def get_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors='pt')
    outputs = model_embeddings(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

In [49]:
def combination_of_probability(p1, p2):
    p1 = p1[0]
    p2 = p2[0]
    X2 = -2 * (np.log(p1) + np.log(p2))
    df = 2 * 2
    p_combined = 1 - chi2.cdf(X2, df)

    return p_combined

In [98]:
system_prompt

'Think step by step. Use a line break if and only if after finishing a step. The last line should be the final answer (be as concise as possible on the final answer). For example: Do this \n Do that \n 4'

In [94]:
def build_question_answer_dataset(question_answer_df, n_executions=2):
    question_answer_df = question_answer_df[['problem', 'solution', 'answer']]

    question_list = []
    solution_list = []
    answer_list = []
    answer_p_list = []
    rational_steps_list = []
    score_list = []
    score1_list = []
    score2_list = []
    for i in range(2):
        question = question_answer_df.iloc[i]['problem']
        solution = question_answer_df.iloc[i]['solution']
        answer = question_answer_df.iloc[i]['answer']
        
        solution_emb = get_embedding(solution)
        answer_emb = get_embedding(answer)

        for i in range(n_executions):
            rational_steps, answer_p = model_rational_steps.question_steps_answer(question=question, system_prompt=system_prompt)

            answer_p_emb = get_embedding(answer_p)
            for rational_step in rational_steps:
                rational_step_emb = get_embedding(rational_step)

                similarity_answer_score = cosine_similarity(answer_emb, answer_p_emb)
                similarity_rational_score = cosine_similarity(solution_emb, rational_step_emb)
                combination_score = combination_of_probability(similarity_answer_score.detach().numpy(), 
                                                               similarity_rational_score.detach().numpy())

                question_list.append(question)
                rational_steps_list.append(rational_step)
                score_list.append(combination_score)

                score1_list.append(similarity_answer_score.detach().numpy()[0])
                score2_list.append(similarity_rational_score.detach().numpy()[0])

                solution_list.append(solution)
                answer_list.append(answer)
                answer_p_list.append(answer_p)
    
    return pd.DataFrame({
        'question': question_list,
        'rational_steps': rational_steps_list,
        'score': score_list,
        'solution': solution_list,
        'answer': answer_list,
        'answer_p': answer_p_list,
        'score_answer': score1_list,
        'score_rational': score2_list})

In [96]:
rational_score_train = build_question_answer_dataset(question_answer_train, n_executions=2)

In [99]:
rational_score_train

,question,rational_steps,score,solution,answer,answer_p,score_answer,score_rational
0,The United States Postal Service charges an ex...,To determine whether the extra postage is need...,0.923366,We calculate the desired ratio for each envelo...,3,3,1.000000,0.634984
1,The United States Postal Service charges an ex...,"For envelope A, the length-to-height ratio is ...",0.985495,We calculate the desired ratio for each envelo...,3,3,1.000000,0.834583
2,The United States Postal Service charges an ex...,"For envelope B, the length-to-height ratio is ...",0.984537,We calculate the desired ratio for each envelo...,3,3,1.000000,0.829375
3,The United States Postal Service charges an ex...,"For envelope C, the length-to-height ratio is ...",0.986406,We calculate the desired ratio for each envelo...,3,3,1.000000,0.839711
4,The United States Postal Service charges an ex...,"For envelope D, the length-to-height ratio is ...",0.984113,We calculate the desired ratio for each envelo...,3,3,1.000000,0.827127
5,The United States Postal Service charges an ex...,"Therefore, we must pay extra postage for envel...",0.913715,We calculate the desired ratio for each envelo...,3,3,1.000000,0.614472
6,The United States Postal Service charges an ex...,To determine which envelopes require extra pos...,0.536659,We calculate the desired ratio for each envelo...,3,The final answer is: $\boxed{3}$,0.316387,0.661567
7,The United States Postal Service charges an ex...,"For Envelope A: length = 6 inches, height = 4 ...",0.567087,We calculate the desired ratio for each envelo...,3,The final answer is: $\boxed{3}$,0.316387,0.724943
8,The United States Postal Service charges an ex...,"For Envelope B: length = 9 inches, height = 3 ...",0.564911,We calculate the desired ratio for each envelo...,3,The final answer is: $\boxed{3}$,0.316387,0.720281
9,The United States Postal Service charges an ex...,"For Envelope C: length = 6 inches, height = 6 ...",0.566615,We calculate the desired ratio for each envelo...,3,The final answer is: $\boxed{3}$,0.316387,0.723929
